In [ ]:
import cv2
import urllib.request
from PIL import Image
from io import BytesIO

import numpy as np
import pandas as pd
from scipy.linalg import sqrtm

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def mk_matrix(img):
    ''' Convert PIL [img] into RGB and HSV Matrix,
    and put them into the [img] Object.
    '''
    img.matrix = dict()
        
    rgb = np.array(img.convert('RGB'))
    
    hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
    
    img.matrix['RGB'] = rgb
    img.matrix['HSV'] = hsv
    
    return img

In [ ]:
def ravel(d, full=False):
    ''' Ravel the 1st and the 2nd Dimensions,
    only keep 500 Pixels to save Computation Time. 
    '''
    s = d.shape
    n = s[0] * s[1]
    d = d.reshape((n, s[2]))
    if n > 500 and not full:
        return d[range(0, n, int(n/500))]
    else:
        return d

def convert(rgb):
    ''' Convert RGB [rgb] values into #FFFFFF like Color String'''
    
    def n2s(n):
        s = hex(n).replace('x', '')
        return s[-2:]
    
    return '#' + ''.join([n2s(n) for n in rgb])

In [ ]:
def color_space(im):
    ''' Draw Color Space of Image [im] '''
    mk_matrix(im)

    v_rgb = ravel(im.matrix['RGB'])
    df = pd.DataFrame(ravel(im.matrix['RGB']), columns=['R', 'G', 'B'])
    df[['H', 'S', 'V']] = ravel(im.matrix['HSV'])
    df['color'] = [convert(e) for e in v_rgb]

    subplot_titles = ('HS', 'SV', 'HV')
    subplot_pos = ((1, 1), (1, 2), (2, 1))
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)

    for pos, title in zip(subplot_pos, subplot_titles):
        x, y = title
        row, col = pos

        _fig = px.scatter(df, x=x, y=y, title=title)
        _fig.data[0]['marker']['color'] = df['color']

        kwargs = dict(
            row=row,
            col=col
        )

        fig.add_trace(
            _fig.data[0],
            **kwargs
        )

        fig.update_xaxes(title_text=x, **kwargs)
        fig.update_yaxes(title_text=y, **kwargs)
        
    fig1 = px.imshow(im)
    fig.add_trace(
        fig1.data[0],
        row=2, col=2
    )

    fig.update_layout(dict(
        height=800,
        width=900,
        title=url
    ))
    fig.show()


#     fig1.show()

In [ ]:
# Converted Image
url = 'http://localhost:8000/132457451652521207_new.jpg'
# url = 'http://localhost:8000/yosemite-5932x3337-5k-4k-wallpaper-8k-forest-osx-apple-mountains-181.jpg'
req = urllib.request.urlopen(url)
im1 = Image.open(BytesIO(req.read()))
color_space(im1)

In [ ]:
# Target Image
# url = 'http://localhost:8000/DP164837.jpg'
# url = 'http://localhost:8000/DT86.jpg'
url = 'http://localhost:8000/yosemite-5932x3337-5k-4k-wallpaper-8k-forest-osx-apple-mountains-181.jpg'
# url = 'http://localhost:8000/macOS-Sierra-Wallpaper-Macbook-Wallpaper.jpg'
req = urllib.request.urlopen(url)
im2 = Image.open(BytesIO(req.read()))
color_space(im2)

In [ ]:
# Compute Cov of Target Img
hsv = im2.matrix['HSV'].copy()
hsv_ravel = ravel(hsv, full=True)

mean2 = np.mean(hsv_ravel, axis=0)
std2 = np.std(hsv_ravel, axis=0)

print(mean2.shape, hsv.shape, hsv_ravel.shape)

cov = np.cov((hsv_ravel-mean2).transpose())
for j in range(3):
    for k in range(3):
        cov[j, k] /= np.sqrt(std2[j] * std2[k])
        
scov2 = sqrtm(cov)

In [ ]:
# Compute Cov of Converted Img
hsv = im1.matrix['HSV'].copy()
shape = hsv.shape
hsv_ravel = ravel(hsv, full=True)

mean = np.mean(hsv_ravel, axis=0)
std = np.std(hsv_ravel, axis=0)

print(mean.shape, hsv.shape, hsv_ravel.shape)

cov = np.cov((hsv_ravel-mean).transpose())
for j in range(3):
    for k in range(3):
        cov[j, k] /= np.sqrt(std[j] * std[k])

scov = sqrtm(cov)
iscov = np.linalg.inv(scov)
iscov

In [ ]:
# Convert Img
hsv_ravel_new = np.matmul(np.matmul(hsv_ravel - mean, iscov), scov2) + mean2
hsv_new = hsv_ravel_new.astype(np.uint8)

for j in [1, 2]:
    hsv_new[:, j][hsv_new[:, j] > 200] = 0
    
hsv_new

In [ ]:
# Plot Converted Img
rgb = cv2.cvtColor(hsv_new.reshape(shape), cv2.COLOR_HSV2RGB)
im = Image.fromarray(rgb)
color_space(im)
px.imshow(im2).show()